In [ ]:
!pip install mesa --quiet
!pip install ipynb --quiet
!pip install Axelrod --quiet
!pip install SALib --quiet
!pip install names-generator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 KB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.0/192.0 KB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.4/386.4 KB 4.5 MB/s eta 0:00:00


In [ ]:
import json
import math
import time
import tqdm
import shutil
import random
import numpy as np
from enum import Enum
import names_generator
from pathlib import Path
from copy import deepcopy
from datetime import datetime
from collections import defaultdict
from typing import List, Type, Tuple

import mesa
from mesa import Agent
from mesa import Model
from mesa.datacollection import DataCollector
from mesa.time import BaseScheduler

import os
import pandas as pd
import matplotlib.pyplot as plt
from itertools import combinations, product

In [ ]:
def softmax(x: np.ndarray, lambda_: float) -> np.ndarray:
    """
    Calculate softmax of x, weighted by lambda.
    This is logit equlibrium function https://en.wikipedia.org/wiki/Quantal_response_equilibrium#Logit_equilibrium
    This function is implemented so it's numerically stable.
    :param x: estimated payoffs for each action
    :param lambda_: rationality factor. 0 - random
    :return: computed softmax
    """
    x = x * lambda_
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)


def sample_action(utilities: np.ndarray, possible_actions: Type[Enum], lambda_: float) -> Enum:
    """
    Sample action taking into considerations estimated utilities per each action.
    :param utilities: per each action estimated utility will be treated as distribution
    :param possible_actions: possible actions for this agent
    :param lambda_: rationality factor. 0 - random
    :return: action
    """
    distribution = softmax(utilities, lambda_)
    action = possible_actions(np.argmax(np.random.multinomial(1, distribution)))
    return action


class CitizenActions(Enum):
    """
    Actions possible for citizen
    """
    accept_complain = 0
    accept_silent = 1
    reject_complain = 2
    reject_silent = 3


class CopActions(Enum):
    """
    Actions possible for cop
    """
    bribe = 0
    not_bribe = 1


class CopMemoryInitial(Enum):
    """
    The value for the Accepting bribe memory initialization. Depending on how Cop is we have different vals.
    """
    Corrupt = 1.0
    Indifferent = 0.5
    Honest = 0.0


class CitizenMemoryInitial(Enum):
    """
    The value for how succesful is complaining. Depending on the system, citizen will have different experiences. Assuming that if system is honest they had nice memories.
    """
    Corrupt = 0.0
    Indifferent = 0.5
    Honest = 1.0

In [ ]:
class Corruption(Model):
    def __init__(self,
                 num_citizens=2500,
                 num_cops=100,
                 team_size=10,
                 rationality_of_agents=10,  # 0 is random totally
                 jail_time=4,
                 prob_of_prosecution=0.7,  # ground truth
                 memory_size=10,
                 fine_amount=1.,  # don't change this in sensitivity analysis
                 cost_complain=0.4,
                 penalty_citizen_prosecution=0.,
                 jail_cost_factor=0.5,
                 # jail cost and jail_time should somehow relate to each other I think, but don't know how exactly
                 cost_accept_mean_std=(0.2, 0.05),
                 citizen_complain_memory_discount_factor=0.5,
                 bribe_amount=0.5,
                 moral_commitment_mean_std=(0.25, 0.1),
                 initial_time_left_in_jail=0,  # don't think it's worth to change that
                 initial_indifferent_corruption_honest_rate=(1.0, 0.0, 0.0),
                 corruption_among_teams_spread=1.0,
                 # rate of teams that should be getting the corrupted cops. 1 - all teams have the same amount(+-1 cop ofc)
                 logger: bool = True,
                 test_params=None,  # parameter that are tested in an experiment and should be saved in the file name
                 ):

        super().__init__()

        if logger:
            if test_params is not None:

                self.experiment_name = ""
                dir_name = ""
                for (param_name, param_value) in test_params.items():
                    setting = param_name + "_" + str(param_value)
                    self.experiment_name += setting + "-"
                    dir_name += "-" + param_name
                self.data_dir = Path("results" + dir_name)
                # remove last dash
                self.experiment_name = self.experiment_name[:-1]
            else:
                # Create a random name for experiment and add date
                now = datetime.now()  # current date and time
                self.experiment_name = names_generator.generate_name() + "_" + now.strftime("%d_%m_%H_%M")
                self.data_dir = Path("results/")
            #print("Saving at: ", str(self.data_dir), "Experiment name: ", self.experiment_name)


        # saving everything, then it can be logged
        self.bribe_amount = bribe_amount
        self.initial_time_left_in_jail = initial_time_left_in_jail
        self.cost_accept_mean_std = cost_accept_mean_std
        self.moral_commitment_mean_std = moral_commitment_mean_std
        self.citizen_complain_memory_discount_factor = citizen_complain_memory_discount_factor
        self.prob_of_prosecution = prob_of_prosecution
        self.memory_size = memory_size
        self.cost_complain = cost_complain
        self.penalty_citizen_prosecution = penalty_citizen_prosecution
        self.fine_amount = fine_amount
        self.rationality_of_agents = rationality_of_agents
        self.num_citizens = num_citizens
        self.num_cops = num_cops
        assert self.num_cops <= self.num_citizens, "There should be more citizens than cops!"
        # cops calculations
        self.num_indifferent_cops = int(initial_indifferent_corruption_honest_rate[0] * num_cops)
        self.num_corrupted_cops = int(initial_indifferent_corruption_honest_rate[1] * num_cops)
        self.num_honest_cops = int(initial_indifferent_corruption_honest_rate[2] * num_cops)
        self.num_honest_cops += self.num_cops - (self.num_corrupted_cops + self.num_honest_cops + self.num_indifferent_cops)

        # citizen calculations
        self.num_indifferent_citizens = int(initial_indifferent_corruption_honest_rate[0] * num_citizens)
        self.num_corrupted_citizens = int(initial_indifferent_corruption_honest_rate[1] * num_citizens)
        self.num_honest_citizens = int(initial_indifferent_corruption_honest_rate[2] * num_citizens)
        self.num_honest_citizens += self.num_citizens - (
                self.num_corrupted_citizens + self.num_honest_citizens + self.num_indifferent_citizens)

        self.team_size = team_size
        assert self.num_cops % self.team_size == 0, \
            f"You need to set num of cops to be dividable by team size. Each team should have the same size! Your num_cops: {self.num_cops}, teamsize: {self.team_size}"
        self.number_of_teams = math.ceil(self.num_cops / self.team_size)

        # how many iterations cop is inactive
        self.jail_time = jail_time
        # actual cost that cop takes into consideration in the utility function
        self.jail_cost_factor = jail_cost_factor
        self.jail_cost = self.jail_cost_factor * jail_time

        assert sum(
            [rate for rate in initial_indifferent_corruption_honest_rate]) == 1.0, "Distribution should sum up to 1."

        # Checking if the value isn't to small, if it is set it to minimum
        self.corruption_among_teams_spread = corruption_among_teams_spread
        self.number_of_corrupted_teams = math.ceil(max(corruption_among_teams_spread * self.number_of_teams,
                                                       initial_indifferent_corruption_honest_rate[
                                                           1] * num_cops / self.team_size))
        # Initialise schedulers
        self.schedule = BaseScheduler(self)
        self.schedule_Cop = BaseScheduler(self)
        self.init_agents()
        # Data collector to be able to save the data
        self.datacollector = self.get_server_data_collector()

        # Divide the cops over a network of teams
        self.create_network()

        self.datacollector.collect(self)
        

        self.logger = logger
        # if self.logger:
        #     # Should be after all initializations as it saves all params!
        #     self.init_logger()

    def step(self):
        self.cops_playing = [cop for cop in self.schedule_Cop.agents if cop.time_left_in_jail == 0]
        self.citizens_playing = random.sample(self.schedule.agents, len(self.cops_playing))

        self.schedule.step()
        self.schedule_Cop.step()

        self.datacollector.collect(self)
        self.update_network()
        # if self.logger:
        #     self.log_data(self.schedule.steps)

    def init_agents(self):
        # Add agents to schedulers
        for i in range(self.num_citizens):

            # citizen_initial_prone_to_complain = citizen_initial_prone_to_complain
            if i < self.num_indifferent_citizens:
                # indifferent
                citizen_initial_prone_to_complain = CitizenMemoryInitial.Indifferent.value
            elif i < self.num_indifferent_citizens + self.num_corrupted_citizens:
                citizen_initial_prone_to_complain = CitizenMemoryInitial.Corrupt.value
            else:
                citizen_initial_prone_to_complain = CitizenMemoryInitial.Honest.value

            citizen = Citizen(i,
                              self,
                              cost_accept_mean_std=self.cost_accept_mean_std,
                              prone_to_complain=citizen_initial_prone_to_complain,
                              complain_memory_discount_factor=self.citizen_complain_memory_discount_factor)
            self.schedule.add(citizen)
        # needed for assigning to teams
        self.lookup_corrupt_cops = defaultdict(list)
        for i in range(self.num_cops):
            if i < self.num_indifferent_cops:
                # indifferent
                accepted_bribe_memory_initial = CopMemoryInitial.Indifferent.value
            elif i < self.num_indifferent_cops + self.num_honest_cops:
                accepted_bribe_memory_initial = CopMemoryInitial.Honest.value
            else:
                accepted_bribe_memory_initial = CopMemoryInitial.Corrupt.value

            cop = Cop(i,
                      self,
                      time_left_in_jail=self.initial_time_left_in_jail,
                      accepted_bribe_memory_size=self.memory_size,
                      bribe_amount=self.bribe_amount,
                      moral_commitment_mean_std=self.moral_commitment_mean_std,
                      accepted_bribe_memory_initial=accepted_bribe_memory_initial)
            self.schedule_Cop.add(cop)
            self.lookup_corrupt_cops[
                "corrupt" if accepted_bribe_memory_initial == CopMemoryInitial.Corrupt.value else "other"].append(i)

    def get_citizen(self):
        """
        Get the citizen chosen citizens and give them to the cop.
        :return: citizen
        """
        citizen = random.sample(self.citizens_playing, 1)[0]
        # remove the citizen so they don't get caught twice in the same iteration
        self.citizens_playing.remove(citizen)
        return citizen

    def create_network(self):
        """
        Create network of police officers. They form not intersecting groups of team_size.
        The amount of bribing cops depend on the `initial_indifferent_corruption_honest_rate`
         and how many in each team `corruption_among_teams_spread`.
        NOTE: indifferent and honest cops are treated the same here. First corrupted cops are allocated and then non-corrupt.
        They're allocated randomly. You can set indifferent rate or the honest rate to 0 and then be sure.
        """

        # depending on this check rate of corruption in the team, maybe if none do it totally randomly?
        if self.number_of_corrupted_teams == 0:
            corrupt_cop_per_team = 0
            surplas_modulo = 0
        else:
            corrupt_cop_per_team = int(self.num_corrupted_cops / self.number_of_corrupted_teams)
            surplas_modulo = self.num_corrupted_cops % self.number_of_corrupted_teams
        #corrupt_cop_per_team = int(self.num_corrupted_cops / self.number_of_corrupted_teams)
        #  some teams might have to take the additional cops
        # surplas_modulo = self.num_corrupted_cops % self.number_of_corrupted_teams

        # Initialise the dictionaries to convert the cop_id to team name, and to convert team name to #cops in jail
        self.id_team = defaultdict(str)
        self.team_jailed = defaultdict(int)
        random.shuffle(self.lookup_corrupt_cops["other"])
        # Corrupted teams first
        for team_number in range(self.number_of_corrupted_teams):
            team_name = "team_" + str(team_number)
            self.team_jailed[team_name] = 0

            number_of_corrupt_cops_in_this_team = corrupt_cop_per_team + (1 if team_number < surplas_modulo else 0)
            # First allocate corrupted cops
            for corrupted_cop in range(number_of_corrupt_cops_in_this_team):
                cop_id = self.lookup_corrupt_cops["corrupt"].pop()
                self.id_team[cop_id] = team_name
            # Allocate not corrupted cops
            for other_cops in range(self.team_size - number_of_corrupt_cops_in_this_team):
                cop_id = self.lookup_corrupt_cops["other"].pop()
                self.id_team[cop_id] = team_name
        # Not Corrupted teams

        for team_number in range(self.number_of_corrupted_teams, self.number_of_teams):
            team_name = "team_" + str(team_number)
            self.team_jailed[team_name] = 0
            # Allocate not corrupted cops
            for other_cops in range(self.team_size):
                # random because some are indifferent and some are honest
                indx = random.randint(0, len(self.lookup_corrupt_cops["other"]) - 1)
                cop_id = self.lookup_corrupt_cops["other"].pop(indx)
                self.id_team[cop_id] = team_name

    def num_active_citizens(self):
        return sum([1 for cit in self.schedule.agents if
                    cit.action is not None])

    def update_network(self):
        """
        Update the current jail rate for each team.
        """
        # Reset the #cops in jail for each team
        for team in self.team_jailed:
            self.team_jailed[team] = 0

        # Update the #cops in jail for each team
        for cop in self.schedule_Cop.agents:
            team = self.id_team[cop.unique_id]
            self.team_jailed[team] += 1 if cop.time_left_in_jail > 0 else 0

    def init_logger(self):
        """
        Logs data in the beginning. Model params and each agent params. Saves it self.log_path at init_params key.
        """

        self.data_dir.mkdir(exist_ok=True)
        self.log_path = Path(self.data_dir, self.experiment_name + '.json')
        try:
            os.remove(self.log_path)
        except:
            pass
        name = 'iteration_0'
        log_dict = self.get_log_data(name)

        with open(self.log_path, 'a') as f:
            json.dump(log_dict, f)
            f.write(os.linesep)

    def log_data(self, step):
        """
        Logs data in each step. Model params and each agent params. Saves it self.log_path at iteration_step key.
        :param step: current iteration
        """
        name = 'iteration_' + str(step)
        log_dict = self.get_log_data(name)

        with open(self.log_path, 'a') as f:
            json.dump(log_dict, f)
            f.write(os.linesep)

    def get_log_data(self, name: str) -> dict:
        """
        Collects data from class fields, throws away unnecessary fields or such that are not easily serializable.
        :return: dict with data
        """
        log_dict = defaultdict(dict)
        log_dict[name] = deepcopy(vars(self))
        log_dict[name].pop('random', None)
        log_dict[name].pop('data_dir', None)
        log_dict[name].pop('running', None)
        log_dict[name].pop('current_id', None)
        log_dict[name].pop('experiment_name', None)
        log_dict[name].pop('log_path', None)
        log_dict[name].pop('schedule', None)
        log_dict[name].pop('schedule_Cop', None)
        log_dict[name].pop('datacollector', None)
        log_dict[name].pop('lookup_corrupt_cops', None)
        log_dict[name].pop('citizens_playing', None)
        log_dict[name].pop('cops_playing', None)
        log_dict[name].pop('logger', None)
        if 'iteration_0' != name:
            # I'm removing those that shouldn't be changed in later steps or it wouldn't change anything if they were
            # rest I left just in case
            log_dict[name].pop('_seed', None)
            log_dict[name].pop('bribe_amount', None)
            log_dict[name].pop('initial_time_left_in_jail', None)
            log_dict[name].pop('cost_accept_mean_std', None)
            log_dict[name].pop('moral_commitment_mean_std', None)
            log_dict[name].pop('fine_amount', None)
            log_dict[name].pop('num_citizens', None)
            log_dict[name].pop('num_cops', None)
            log_dict[name].pop('num_indifferent_cops', None)
            log_dict[name].pop('num_corrupted_cops', None)
            log_dict[name].pop('num_honest_cops', None)
            log_dict[name].pop('num_indifferent_citizens', None)
            log_dict[name].pop('num_corrupted_citizens', None)
            log_dict[name].pop('num_honest_citizens', None)
            log_dict[name].pop('number_of_teams', None)
            log_dict[name].pop('corruption_among_teams_spread', None)
            log_dict[name].pop('number_of_corrupted_teams', None)
            log_dict[name].pop('citizen_complain_memory_discount_factor', None)
            log_dict[name].pop('prob_of_prosecution', None)
            log_dict[name].pop('memory_size', None)
            log_dict[name].pop('cost_complain', None)
            log_dict[name].pop('penalty_citizen_prosecution', None)
            log_dict[name].pop('rationality_of_agents', None)
            log_dict[name].pop('team_size', None)
            log_dict[name].pop('jail_time', None)
            log_dict[name].pop('jail_cost_factor', None)
            log_dict[name].pop('jail_cost', None)
            log_dict[name].pop('id_team', None)

        # add agents stats
        log_dict[name]['citizens'] = {}
        log_dict[name]['cops'] = {}

        # data_cit = [cit.log_data() for cit in self.schedule.agents]
        # action_mean = sum(1 for d in data_cit if d['action'] == 'bribe') / self.num_citizens
        


        for cit in self.schedule.agents:
            log_dict[name]['citizens'][cit.unique_id] = cit.log_data()
            if 'iteration_0' != name:
                log_dict[name]['citizens'][cit.unique_id].pop('cost_accept', None)

        for cop in self.schedule_Cop.agents:
            log_dict[name]['cops'][cop.unique_id] = cop.log_data()
            if 'iteration_0' != name:
                log_dict[name]['cops'][cop.unique_id].pop('moral_commitment', None)
        
        return log_dict

    def get_server_data_collector(self):
        return DataCollector(
            {"Prison_Count": lambda m: sum([1 for cop in self.schedule_Cop.agents if
                                            cop.time_left_in_jail > 0]),
             "Bribing": lambda m: sum([1 for cop in self.schedule_Cop.agents if
                                       cop.action == CopActions.bribe]),
             "Not_Bribing": lambda m: sum([1 for cop in self.schedule_Cop.agents if
                                       cop.action == CopActions.not_bribe]),
             "AcceptComplain": lambda m: sum([1 for cit in self.schedule.agents if
                                                cit.action == CitizenActions.accept_complain]),
             "Reject_Complain": lambda m: sum([1 for cit in self.schedule.agents if
                                                cit.action == CitizenActions.reject_complain]),
             "Accept_Silent": lambda m: sum([1 for cit in self.schedule.agents if
                                                cit.action == CitizenActions.accept_silent]),
             "Reject_Silent": lambda m: sum([1 for cit in self.schedule.agents if
                                                cit.action == CitizenActions.reject_silent]),
             "Total Complain": lambda m: sum([1 for cit in self.schedule.agents if
                                                cit.action == CitizenActions.accept_complain or cit.action == CitizenActions.reject_complain]),
             "Total Accept": lambda m: sum([1 for cit in self.schedule.agents if
                                            cit.action == CitizenActions.accept_complain or cit.action == CitizenActions.accept_silent]),
             "Estimated_Prob_Accept": lambda m: float(np.mean([cop.estimated_prob_accept for cop in self.schedule_Cop.agents])),
             "Estimated_Prob_Caught": lambda m: float(np.mean([cop.approximate_prob_caught() for cop in self.schedule_Cop.agents])),
             "Complain_Memory": lambda m: float(np.mean([cit.complain_memory for cit in self.schedule.agents]))})

In [ ]:
class Citizen(Agent):
    def __init__(self,
                 unique_id,
                 model,
                 cost_accept_mean_std: Tuple[float, float],
                 complain_memory_discount_factor: float,
                 prone_to_complain: float = 0.5,
                 first_action: str = None
                 ):
        super().__init__(unique_id, model)

        self.action = first_action

        # initialize moral costs
        self.cost_accept = np.random.normal(loc=cost_accept_mean_std[0], scale=cost_accept_mean_std[1])

        # Initialize memory, complain_memory ==0.5 means that the beginning state is being indifferent
        self.complain_memory_accumulated_weights = 1
        self.complain_memory = prone_to_complain
        # 0 is easily forgetting, 1 all events important the same
        self.discount_factor = complain_memory_discount_factor

        self.possible_actions = CitizenActions

    def do_action(self, bribe_amount):
        prob_success_complain = self.approximate_prob_successful_complain()

        # complain_reward = bribe to make the # of params smaller
        utility_accept_complain = -bribe_amount + prob_success_complain * (
                bribe_amount - self.model.penalty_citizen_prosecution) - self.model.cost_complain - self.cost_accept
        utility_accept_silent = -bribe_amount - self.cost_accept
        utility_reject_complain = -self.model.fine_amount - self.model.cost_complain
        utility_reject_silent = -self.model.fine_amount

        utilities = np.array([utility_accept_complain,
                              utility_accept_silent,
                              utility_reject_complain,
                              utility_reject_silent])

        self.action = sample_action(utilities, self.possible_actions, self.model.rationality_of_agents)

    def step(self):
        self.action = None

    def approximate_prob_successful_complain(self):
        """
        Estimate how successful complain will be.
        :return: estimated probability of cop being prosecuted.
        """
        return self.complain_memory

    def update_successful_complain_memory(self, update):
        """
        Updates running, discounted average. This way doesn't require remembering each event.
        Discount factor should be in [0,1] range.
         0 is when only last experience is important. 1 - all experiences are weighted the same
        :param update: complain event result, 0 - cop is not caught, 1 - cop is caught
        """
        old_complain_memory_sum_weights = self.complain_memory_accumulated_weights
        self.complain_memory_accumulated_weights = self.complain_memory_accumulated_weights * self.discount_factor + 1
        old_mean_rate = old_complain_memory_sum_weights / self.complain_memory_accumulated_weights

        self.complain_memory = self.discount_factor * old_mean_rate * self.complain_memory + update / self.complain_memory_accumulated_weights
        assert self.complain_memory <= 1.0 or self.complain_memory >= 0.0, (
                "Complain memory is out of proper range! " + str(self.complain_memory))

    def log_data(self) -> dict:
        """
        Creates a dictionary with all params of this agent
        :return: dict with results
        """

        data = {'action': self.action if self.action is None else self.action.name,
                'cost_accept': self.cost_accept,
                'complain_memory': self.complain_memory}
        return data.copy()


class Cop(Agent):
    def __init__(self, unique_id,
                 model,
                 time_left_in_jail: int,
                 accepted_bribe_memory_size: int,
                 bribe_amount: float,
                 moral_commitment_mean_std: Tuple[float, float],
                 first_action: str = None,
                 accepted_bribe_memory_initial: float = 0.5):
        super().__init__(unique_id, model)

        self.action = first_action
        self.bribe_amount = bribe_amount

        self.time_left_in_jail = time_left_in_jail
        self.accepted_bribe_memory_size = accepted_bribe_memory_size
        self.accepted_bribe_memory = [accepted_bribe_memory_initial] * accepted_bribe_memory_size
        # for stats
        self.estimated_prob_accept = self.approximate_prob_accept()

        self.moral_commitment = np.random.normal(loc=moral_commitment_mean_std[0], scale=moral_commitment_mean_std[1])

        self.possible_actions = CopActions

    def step(self):
        self.action = None
        # Check whether this cop can play this round
        play = self.validate_play()

        # If cop can play, chose a random citizen from the available citizens and sample an action for the cop
        if play:
            self.do_action()
            citizen = self.model.get_citizen()

            # If the cop bribes, sample an action for the citizen
            if self.action == CopActions.bribe:
                citizen.do_action(self.bribe_amount)

                # If the citizen complains, give the cop a jail time based on the ground truth of the probability of getting caught
                if citizen.action in [CitizenActions.accept_complain, CitizenActions.reject_complain]:
                    if np.random.multinomial(1,
                                             [self.model.prob_of_prosecution,
                                              1 - self.model.prob_of_prosecution])[0] == 1:
                        # complain succesful -> cop goes to jail
                        self.time_left_in_jail = self.model.jail_time
                        citizen.update_successful_complain_memory(1)
                    else:
                        # complain failed, citizen remembers that
                        citizen.update_successful_complain_memory(0)
                # If the citizen accepts to bribe, update this in the memory for the cop
                if citizen.action in [CitizenActions.accept_complain, CitizenActions.accept_silent]:
                    self.update_accepting_bribe_memory(1)

                # If the citizen rejects to bribe, update this in the memory for the cop
                if citizen.action in [CitizenActions.reject_complain, CitizenActions.reject_silent]:
                    self.update_accepting_bribe_memory(0)


        # If the cop cannot play, check whether the cop is in jail and if the cop is in jail, reduce their sentence by 1
        elif self.time_left_in_jail > 0:
            self.time_left_in_jail -= 1

    def validate_play(self):
        """
        Checks if the cop is allowed to play. They are allowed if they're not in jail. This is checked in model
        :return:True if allowed to play
        """
        return True if self in self.model.cops_playing else False

    def do_action(self):
        """
        Cop is making an action based on utilities. The sampled action is then saved in the self.action field.
        """

        approx_prob_caught = self.approximate_prob_caught()
        approx_prob_accept = self.approximate_prob_accept()

        # Calculate expected utilities for each action
        utility_bribe = (1 - approx_prob_caught) * approx_prob_accept * self.bribe_amount - approx_prob_caught * self.model.jail_cost
        utility_not_bribe = self.moral_commitment

        utilities = np.array([utility_bribe, utility_not_bribe])

        self.action = sample_action(utilities, self.possible_actions, self.model.rationality_of_agents)

    def approximate_prob_caught(self):
        """
        This function checks how many cops in the network/group are currently in jail. This rate is the estimated probability of probability of prosecution
        :return: estimated probability of getting caught, 0 to 1
        """
        team = self.model.id_team[self.unique_id]
        m = self.model.team_jailed[team]
        self.estimated_prob_caught = m / self.model.team_size
        return self.estimated_prob_caught

    def update_accepting_bribe_memory(self, update):
        """
        Writes the information about last bribing attempt being successful. Keeps the memory in certain size.
        :param update: last bribing attempt result. 0 - not successful, 1 - successful
        """

        self.accepted_bribe_memory.append(update)
        if len(self.accepted_bribe_memory) > self.accepted_bribe_memory_size:
            self.accepted_bribe_memory.pop(0)

    def approximate_prob_accept(self):
        """
        Takes the average of the attempts.
        :return: estimated probability of accepting the bribe by a citizen
        """
        # saving it here to have it in the logged data
        self.estimated_prob_accept = sum(self.accepted_bribe_memory) / self.accepted_bribe_memory_size
        return self.estimated_prob_accept

    def log_data(self) -> dict:
        """
        Creates a dictionary with all params of this agent
        :return: dict with results
        """

        data = {'action': self.action if self.action is None else self.action.name,
                'time_left_in_jail': self.time_left_in_jail,
                # 'accepted_bribe_memory': self.accepted_bribe_memory.copy(),
                'estimated_prob_accept': self.estimated_prob_accept,
                'moral_commitment': self.moral_commitment,
                'approximated_prob_caught': self.approximate_prob_caught()}
        return data

In [ ]:
class Experiment():

    def __init__(self,
                 data_params,
                 default_params,
                 max_steps,
                 cop_params,
                 cit_params,
                 folder):

        self.data_params = data_params
        self.default_params = default_params
        self.max_steps = max_steps
        self.cop_params = cop_params
        self.cit_params = cit_params
        self.folder = folder

        self.pair_combinations = list(combinations(self.data_params.keys(), 2))

        try:
            os.mkdir(self.folder)
        except:
            pass

    def create_data(self, init_name, init_values):

        try:
            os.mkdir(self.folder + "/" + init_name)
        except:
            pass

        documentation = []

        for param1, param2 in self.pair_combinations:
            range_param1, range_param2 = self.data_params[param1], self.data_params[param2]
            data_combinations = list(product(range_param1, range_param2))

            experiment = []

            for setting_param1, setting_param2 in tqdm.tqdm(data_combinations):
                test_params = self.default_params.copy()
                test_params[param1], test_params[param2] = setting_param1, setting_param2
                test_params["initial_indifferent_corruption_honest_rate"] = init_values

                model = Corruption(test_params= {param1: setting_param1, param2: setting_param2}, *test_params.values())

                start = time.time()
                for _ in range(self.max_steps):
                    model.step()
                end = time.time()
                experiment.append(model.datacollector.get_model_vars_dataframe())
            
            experiment_path = self.folder + "/" + init_name + "/" + param1 + "-" + param2 + ".csv"
            pd.concat(experiment).to_csv(experiment_path, index=False)
            documentation.append([experiment_path, param1, param2, range_param1, range_param2, self.max_steps, len(data_combinations)])

        pd.DataFrame(documentation, columns=['experiment_path',
                                            'param1',
                                            'param2',
                                            'range_param1',
                                            'range_param2',
                                            'max_steps',
                                            'num_experiments']).to_csv("/content/experiment_documentation_" + init_name + ".csv", index=False)
        shutil.make_archive(init_name, 'zip', self.folder + "/" + init_name)

if __name__ == '__main__':
    num_settings = 8
    data_params = {"team_size": [1, 2, 5, 10, 20, 25, 50, 100][0:num_settings],
                   "corruption_among_teams_spread": np.linspace(0.0, 0.99, num=num_settings).tolist(),
                   "jail_time": np.arange(1, num_settings+1).tolist(),
                   "prob_of_prosecution": np.linspace(0.01, 0.99, num=num_settings).tolist(),
                   "cost_complain": np.linspace(0.01, 5, num=num_settings).tolist()}

    data_params_T = {"jail_time": np.arange(1, 5).tolist(),
                     "prob_of_prosecution": np.linspace(0.01, 0.99, num=4).tolist()}

    default_params = {"num_citizens": 1000,
                      "num_cops": 100,
                      "team_size": 10,
                      "rationality_of_agents": 10,
                      "jail_time": 4,
                      "prob_of_prosecution": 0.7,
                      "memory_size": 10,
                      "fine_amount": 1.,
                      "cost_complain": 0.4,
                      "penalty_citizen_prosecution": 0.,
                      "jail_cost_factor": 0.5,
                      "cost_accept_mean_std": (0.2, 0.05),
                      "citizen_complain_memory_discount_factor": 0.5,
                      "bribe_amount": 0.5,
                      "moral_commitment_mean_std": (0.25, 0.1),
                      "initial_time_left_in_jail": 0,
                      "initial_indifferent_corruption_honest_rate": (1.0, 0.0, 0.0),
                      "corruption_among_teams_spread": 1.0,
                      "logger": True}

    cop_params = ['action', 'time_left_in_jail', 'estimated_prob_accept', 'approximated_prob_caught']
    cit_params = ['action', 'complain_memory']

    initialisations = {'indifferent': (1.0, 0.0, 0.0),
                       'corrupt': (0.0, 1.0, 0.0),
                       'honest': (0.0, 0.0, 1.0)}

    experiment = Experiment(data_params, default_params, 250, cop_params, cit_params, '/content/results')
    for init_name, init_values in initialisations.items():
        experiment.create_data(init_name, init_values)

In [ ]:
def read_experiment(documentation_path, vis_param, mode, window):

    initialisation = documentation_path.split('.')[0].split('_')[-1]

    experiments = pd.read_csv(documentation_path, delimiter = ',')
    columns = experiments.columns
    experiments_array = experiments.to_numpy()

    for experiment in experiments_array:
        experiment_path, param1, param2, range_param1, range_param2, max_steps, num_experiments = experiment
        experiment_data = pd.read_csv(experiment_path, delimiter = ',')

        range_param1 = json.loads(range_param1)
        range_param2 = json.loads(range_param2)

        #visualize_grid(experiment_data, vis_param, param1, param2, range_param1, range_param2, mode, num_experiments, initialisation)
        visualize_graph(experiment_data, vis_param, param1, param2, range_param1, range_param2, num_experiments, window, initialisation)

def visualize_graph(experiment_data, vis_param, param1, param2, range_param1, range_param2, num_experiments, window, initialisation):

    X = list(range(len(experiment_data)//num_experiments))
    Y = []

    for bottom in range(0, len(experiment_data), len(experiment_data)//num_experiments):
        top = bottom + len(experiment_data)//num_experiments
        slice_ = experiment_data.iloc[bottom: top]
        slice_2 =  slice_.rolling(window=window).mean()
        Y.append(slice_2["Bribing"].values)
    
    for y in Y:
        plt.plot(X, y)

    plt.xlabel('Iteration')
    plt.ylabel(str(vis_param))
    plt.title("Running average with window=" + str(window) + '\n' + " (" + param1 + " and " + param2 + ")")
    plt.savefig("/content/figures/" + initialisation + "_graph_" + param1 + "_" + param2  + ".png", bbox_inches='tight')
    plt.show()

def visualize_grid(experiment_data, vis_param, param1, param2, range_param1, range_param2, mode, num_experiments, initialisation):

    z = []

    for bottom in range(0, len(experiment_data), len(experiment_data)//num_experiments):
        top = bottom + len(experiment_data)//num_experiments
        slice_ = experiment_data.iloc[bottom: top]
        num_active = (slice_["Bribing"] + slice_["Not_Bribing"])
        if mode == 'mean':
            param_freq = np.mean(slice_[vis_param] / num_active)
        if mode == 'percent':
            param_freq = np.mean(slice_[vis_param].iloc[int(num_experiments*0.1):] / num_active.iloc[int(num_experiments*0.1):])
        if mode == 'median':
            param_freq = np.median(slice_[vis_param] / num_active)
        
        z.append(round(param_freq, 2))

    z = np.array(np.split(np.array(z), len(range_param1)))
    
    fig, ax = plt.subplots()
    im = ax.imshow(z)

    ax.set_xticks(np.arange(len(range_param2)))
    ax.set_yticks(np.arange(len(range_param1)))
    ax.set_xticklabels([round(item, 2) for item in range_param2])
    ax.set_yticklabels([round(item, 2) for item in range_param1])

    plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")

    for i in range(len(range_param1)):
        for j in range(len(range_param2)):
            text = ax.text(j, i, z[i, j], ha="center", va="center", color="b")

    ax.set_title(vis_param)
    ax.set_xlabel(param2)
    ax.set_ylabel(param1)

    fig.tight_layout()

    try:
        os.mkdir("/content/figures") 
    except:
        pass

    plt.savefig("/content/figures/" + initialisation + "_grid_" + param1 + "_" + param2  + ".png", bbox_inches='tight')
    plt.show()

read_experiment('/content/experiment_documentation_corrupt.csv', 'Bribing', 'mean', 20)